In [1]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
from netpixi.integration.gt import *
import pandas as pd
import numpy as np
import csv
import ast

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [2]:
df = pd.read_csv('./Video_Games_Sales_as_at_22_Dec_2016.csv')
df.dropna(subset=['Publisher'], inplace=True)
df

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.groupby('Genre')['Name'].count().sort_values(ascending=False)

Genre
Action          3364
Sports          2345
Misc            1723
Role-Playing    1499
Shooter         1321
Adventure       1299
Racing          1248
Platform         887
Simulation       870
Fighting         847
Strategy         680
Puzzle           580
Name: Name, dtype: int64

In [4]:
df_grouped_Genre = df.groupby(['Genre']).agg({'Global_Sales': np.mean})
df_grouped_Genre.sort_values('Global_Sales', ascending=False)

,Global_Sales
Genre,
Platform,0.933033
Shooter,0.797048
Role-Playing,0.623316
Racing,0.584046
Sports,0.567343
Fighting,0.524522
Action,0.518564
Misc,0.461619
Simulation,0.448483


In [5]:
sports_mean_sales = df_grouped_Genre.loc['Sports']['Global_Sales']
action_mean_sales = df_grouped_Genre.loc['Action']['Global_Sales'] 
rpg_mean_sales = df_grouped_Genre.loc['Role-Playing']['Global_Sales'] 

In [6]:
def segregate_genre_to_list(df, genre, df_list, threshold=None):
    seg_df = df[df['Genre'] == genre]
    if threshold:
        seg_df = seg_df[seg_df['Global_Sales'] > threshold]
    df_list.append(seg_df)

In [7]:
sorted_years = df['Year_of_Release'].unique()
sorted_years.sort()
print(sorted_years)

[1980. 1981. 1982. 1983. 1984. 1985. 1986. 1987. 1988. 1989. 1990. 1991.
 1992. 1993. 1994. 1995. 1996. 1997. 1998. 1999. 2000. 2001. 2002. 2003.
 2004. 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014. 2015.
 2016. 2017. 2020.   nan]


In [8]:
df_list = []
sport_df_list = []
action_df_list = []
rpg_df_list = []
for i in range(0, len(sorted_years), 5):
    df_list.append(df[df['Year_of_Release'].isin(sorted_years[i:i+5])])

for db in df_list:
    segregate_genre_to_list(db, 'Sports', sport_df_list, threshold=sports_mean_sales)
    segregate_genre_to_list(db, 'Action', action_df_list, threshold=action_mean_sales)
    segregate_genre_to_list(db, 'Role-Playing', rpg_df_list, threshold=rpg_mean_sales)

In [9]:
publisher_platforms = []
for db in rpg_df_list:
    publisher_platforms.append(db.groupby('Platform')['Publisher'].unique().to_dict())

In [10]:
action_publishers = df[df['Genre'] == 'Action']["Publisher"].unique()
rpg_publishers = df[df['Genre'] == 'Role-Playing']["Publisher"].unique()
sports_publishers = df[df['Genre'] == 'Sports']["Publisher"].unique()

In [11]:
action_graph = Graph(directed=False) # não-dirigido

for publisher in action_publishers:
    action_graph.add_vertex(publisher)

In [12]:
for platform_dict in publisher_platforms:
    for platform, publishers in platform_dict.items():
        for i in range(len(publishers)):
            if i != len(publishers)-1:
                next_vertex = i+1
                while(next_vertex < len(publishers)):
                    if(publishers[i] != publishers[next_vertex]):
                        try:
                            action_graph.add_edge(publishers[i], publishers[next_vertex])
                            #shared_pathogens.get_edge(publishers[i]-1, publishers[next_vertex]-1)['strength'] = 1
                        except ValueError:
                            continue
                            #shared_pathogens.get_edge(publishers[i]-1, publishers[next_vertex]-1)['strength'] += 1
                        finally:    
                            next_vertex += 1
                    else:
                        next_vertex += 1
                    
                    
print('número de arestas de g:', action_graph.num_edges())



número de arestas de g: 156


In [13]:
print('número de vértices de g:', action_graph.num_vertices())

número de vértices de g: 235


In [14]:
gt_save(action_graph, 'action_graph.net.gz')

In [15]:
g = gt_load('action_graph.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'action_sfdp.net.gz')

In [16]:
r = netpixi.render('action_sfdp.net.gz', infinite=True);

In [17]:
rpg_graph = Graph(directed=False) # não-dirigido

for publisher in rpg_publishers:
    rpg_graph.add_vertex(publisher)

In [18]:
for platform_dict in publisher_platforms:
    for platform, publishers in platform_dict.items():
        for i in range(len(publishers)):
            if i != len(publishers)-1:
                next_vertex = i+1
                while(next_vertex < len(publishers)):
                    if(publishers[i] != publishers[next_vertex]):
                        try:
                            rpg_graph.add_edge(publishers[i], publishers[next_vertex])
                            #shared_pathogens.get_edge(publishers[i]-1, publishers[next_vertex]-1)['strength'] = 1
                        except ValueError:
                            continue
                            #shared_pathogens.get_edge(publishers[i]-1, publishers[next_vertex]-1)['strength'] += 1
                        finally:    
                            next_vertex += 1
                    else:
                        next_vertex += 1
                    
                    
print('número de arestas de g:', rpg_graph.num_edges())



número de arestas de g: 207


In [19]:
print('número de vértices de g:', rpg_graph.num_vertices())

número de vértices de g: 160


In [20]:
gt_save(rpg_graph, 'rpg_graph.net.gz')

In [21]:
g = gt_load('rpg_graph.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'rpg_sfdp.net.gz')

In [22]:
r = netpixi.render('rpg_sfdp.net.gz', infinite=True);

In [23]:
sports_graph = Graph(directed=False) # não-dirigido

for publisher in sports_publishers:
    sports_graph.add_vertex(publisher)

In [24]:
for platform_dict in publisher_platforms:
    for platform, publishers in platform_dict.items():
        for i in range(len(publishers)):
            if i != len(publishers)-1:
                next_vertex = i+1
                while(next_vertex < len(publishers)):
                    if(publishers[i] != publishers[next_vertex]):
                        try:
                            sports_graph.add_edge(publishers[i], publishers[next_vertex])
                            #shared_pathogens.get_edge(publishers[i]-1, publishers[next_vertex]-1)['strength'] = 1
                        except ValueError:
                            continue
                            #shared_pathogens.get_edge(publishers[i]-1, publishers[next_vertex]-1)['strength'] += 1
                        finally:    
                            next_vertex += 1
                    else:
                        next_vertex += 1
                    
                    
print('número de arestas de g:', sports_graph.num_edges())



número de arestas de g: 133


In [25]:
print('número de vértices de g:', sports_graph.num_vertices())

número de vértices de g: 164


In [26]:
gt_save(sports_graph, 'sports_graph.net.gz')

In [27]:
g = gt_load('sports_graph.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'sports_sfdp.net.gz')

In [28]:
r = netpixi.render('sports_sfdp.net.gz', infinite=True);